In [3]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string
import csv

In [4]:
def parseLabeledData(path):
    file=open(path, 'r')
    dataList = []
    for line in csv.reader(file):
        if len(line) == 4:
            dataList.append(
                {"asin":line[0], 
                 "question":line[1],
                 "review":line[2],
                 "label":line[3]}
            )     
    return dataList
        

print("Reading labeled data...")
data = parseLabeledData("/Users/Silvia/Desktop/LabelledData.csv")
queries = [d['question'] for d in data]
reviews = [d['review'] for d in data]
labels = [d['label'] for d in data]
print("done")

Reading labeled data...
done


In [6]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line["question"])
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line["reviewText"])
      
    return dataList

print("Reading all reviews & all questions...")
allReviews = parseAllReviews("/Users/Silvia/Downloads/reviews.json")
allQuestions = parseAllQueries("/Users/Silvia/Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = [review for review in allReviews] + [question for question in allQuestions]

print("done")

Reading all reviews & all questions...
done


In [27]:
def findCommonWords():
    allWords = defaultdict(int)
    for r in allReviews.values():
        for review in r:
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                allWords[w] += 1
    
    
    return sorted(allWords, key=lambda x: -allWords[x])[:1000]

In [28]:
commonWords = findCommonWords()

In [101]:
# @param a word whose frequency in the document we are calculating
# @param document a string of a review or a question
# @return the frequency of term in document div length of document

def tf(term, document):
    count = collections.defaultdict(int)
    for word in document.split():
        count[word] += 1

    return count[term]/len(document.split())

In [112]:
def idf(term):
    count = 0         
    for doc in docSet:
        if term in doc.lower():
            count += 1
    return math.log(len(docSet)/(count+1))

In [29]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

In [33]:
def feature(review, question):
    review = review.lower()
    review = ''.join([c for c in review if not (c in string.punctuation)])
    
    featReview = [0]*1000
    
    for term in review.split():
        index = wordToIndex(term)
        if index != -1:
            featReview[index] += 1
            
    question = question.lower()
    question = ''.join([c for c in question if not (c in string.punctuation)])
    
    featQuestion = [0]*1000
    
    for term in question.split():
        index = wordToIndex(term)
        if index != -1:
            featQuestion[index] += 1
            
    return [a*b for a,b in zip(featQuestion,featReview)]

In [93]:
def linearRegression():
    X = [feature(d["review"], d["question"]) for d in data]
    y = [1 if l == "y" else 0 for l in labels]
    
    X_train = X[:len(X)//2]
    y_train = y[:len(y)//2]
    
    X_test = X[len(X)//2:]
    y_test = y[len(y)//2:]
    
    theta,residuals,rank,s = numpy.linalg.lstsq(X_train, y_train, rcond=None)
    
    X_train = numpy.matrix(X_train)
    y_train = numpy.matrix(y_train).T
    theta = numpy.matrix(theta).T
    y_hat = X_train * theta
    
    #print(X_test)
    #print(y_test)
    #print(theta)
    #print(y_hat)
    
    y_bar = numpy.sum(y_train) / len(y_train)
    
    top = numpy.sum(numpy.square(y_train - y_hat))
    bottom = numpy.sum(numpy.square(y_train - y_bar))
    
    #print(y_hat - y_bar)
    
    print(top, bottom, top / bottom)
    
    
    
    
    
    
    
    #MSE = sum(numpy.square(y_hat - y_test)) / len(y_hat)
    #print("Test: ", MSE)
    
    
    #X_train = numpy.matrix(X_train)
    #y_train = numpy.matrix(y_train).T
    #y_hat = X_train * theta
    
    #MSE = sum(numpy.square(y_hat - y_train)) / len(y_hat)
    #print("Train: ", MSE)
    
    
    
linearRegression()
    
    


1.0374532467545183 15.254237288135597 0.0680108239539073


NameError: name 'qa' is not defined